In [ ]:
%%capture
### Installation

!pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

In [ ]:
### Unsloth

from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Vikhrmodels/QVikhr-2.5-1.5B-Instruct-r", # Qwen/Qwen2.5-14B-Instruct-1M  Qwen/Qwen2.5-7B-Instruct-1M  Vikhrmodels/QVikhr-2.5-1.5B-Instruct-r
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
### LORA

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
### Data Prep

from unsloth.chat_templates import get_chat_template
from datasets import Dataset
import pandas as pd

# Получаем базовый шаблон chatml
tokenizer = get_chat_template(
    tokenizer,
    chat_template="chatml",
    map_eos_token=True,
)

# Настраиваем токены
tokenizer.add_special_tokens({
    "additional_special_tokens": [
        "<|im_start|>",
        "<|im_end|>",
    ],
})

# Устанавливаем шаблон Qwen
tokenizer.chat_template = "{% for message in messages %}\n    {{- '<|im_start|>' + message.role + '\n' + message.content  + '<|im_end|>' + '\n' -}}\n{% endfor %}\n{%- if add_generation_prompt %}\n    {{- '<|im_start|>assistant\n' -}}\n{%- endif %}"

def formatting_prompts_func(examples):
    convos = []
    for system, user, assistant in zip(examples["system"], examples["user"], examples["assistant"]):
        messages = [
            {"role": "system", "content": system},
            {"role": "user", "content": user},
            {"role": "assistant", "content": assistant}
        ]
        formatted_text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False
        )
        convos.append(formatted_text)
        
        # Выводим первый пример для отладки
        if len(convos) == 1:
            print("\nПример форматированного текста:")
            print("-" * 50)
            print(formatted_text)
            print("-" * 50)
            
    return {"text": convos}

# Загрузка данных
df = pd.read_parquet("dataset/merged_dataset_hb_cl.parquet")[["system", "user", "assistant"]]
dataset = Dataset.from_pandas(df)
dataset = dataset.map(formatting_prompts_func, batched=True)

In [ ]:
### Train the model

from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1,
        learning_rate = 1e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs_qwen",
        report_to = "none", # Use this for WandB etc
    ),
)

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
### Train Start

trainer_stats = trainer.train()

In [ ]:
### Saving, loading finetuned models

model.save_pretrained("lora_model_tlite")  # Local saving
tokenizer.save_pretrained("lora_model_tlite")